In [1]:
import dores
import pandas as pd
from tqdm.notebook import trange
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', 1000)
import numpy as np
import plotly.graph_objects as go
import datetime

In [2]:
filename = './data/LichteilandGoerree/hp.csv'
dataset = dores.import_data(filename)
dataset['VALUE'] = dataset['VALUE'] / 100
dataset['Tp'] = 5 * (dataset['VALUE']) ** 0.5
dataset.columns = ['DATETIME', 'WAVE HEIGHT', 'WAVE PERIOD']

In [3]:
date_list = np.arange(
    start=datetime.datetime(2010, 1, 1),
    stop=datetime.datetime(2017, 11, 1),
    step=datetime.timedelta(minutes=10)
)

periods = []

for i in trange(300):
    idx = np.random.randint(0, len(date_list))
    startDate = date_list[idx].astype(datetime.datetime)
    sim = dores.DiscreteEventSimulation(start_date=startDate)
    sim.offshore_environment(dataframe=dataset)
    sim.create_entities()
    sim.start_simulation()
    periods.append(sim.get_project_length())

In [30]:
hist = go.Histogram(
    x=periods,
    xbins={'size': 2},
    cumulative={'enabled': True},
    histnorm='probability',
    marker=dict(color='red', line=dict(color='black', width=1)),
    name='cumulative distribution'
)

fig = go.Figure(data=hist)

fig.add_trace(go.Scatter(
    x=dores.cumulative_dist(periods).iloc[::1]['X'],
    y=dores.cumulative_dist(periods).iloc[::1]['EDF'],
    line={'shape': 'spline', 'color': 'blue'},
    name='exceedance probability'
))

fig.update_layout(
    xaxis_title='Estimated construction period [days]',
    yaxis_title='Probability [-]',
    title='Cumulative distribution function ECP',
    plot_bgcolor='white'
)

fig.show()